In [1]:
import domino
from domino.training_sets import TrainingSetClient, model
import pandas as pd
import os

In [9]:
path = '/mnt/code/Test&TrainData/churnTrainingData.csv'

training_df = pd.read_csv(path)
training_df.head()

,custid,dropperc,mins,consecmonths,income,age,churn_Y
0,844336,0.016364,550,28,89.2,45,0
1,146041,0.018349,545,33,54.2,43,0
2,847745,0.018519,378,41,55.3,41,0
3,285565,0.014493,552,32,66.8,31,0
4,754611,0.012132,577,4,87.2,43,0


In [10]:
feature_names = ["custid", "dropperc", "mins", "consecmonths", "income", "age", "churn_Y"]
training_df_subset= training_df[feature_names].copy()

In [11]:
training_df_subset["custid"] = training_df_subset.index + 1

In [12]:
training_df_subset.head()

,custid,dropperc,mins,consecmonths,income,age,churn_Y
0,1,0.016364,550,28,89.2,45,0
1,2,0.018349,545,33,54.2,43,0
2,3,0.018519,378,41,55.3,41,0
3,4,0.014493,552,32,66.8,31,0
4,5,0.012132,577,4,87.2,43,0


In [13]:
tsv = TrainingSetClient.create_training_set_version(
    training_set_name="customer-churn-DMM-demo-",
    df=training_df_subset,
    key_columns=["custid"],
    target_columns=["churn_Y"],
    exclude_columns=[],
    meta={"DMM_meta_data": "0.1"},
    monitoring_meta=model.MonitoringMeta(**{
        "categorical_columns": [],
        "timestamp_columns": [],
        "ordinal_columns": []
    }),
    project_name=str(os.environ.get('DOMINO_PROJECT_OWNER')+os.environ.get('DOMINO_PROJECT_NAME'))
)

print(f"TrainingSetVersion {tsv.training_set_name}:{tsv.number}")

TrainingSetVersion customer-churn-DMM-demo-:1


In [14]:
tsv.all_columns

['custid', 'dropperc', 'mins', 'consecmonths', 'income', 'age', 'churn_Y']